In [37]:
# This notebook is an example of Pandas chaining using lambda functions that I made to tweak 
# a blockchain csv output for tax purposes.  This style of coding in pandas avoids the setting with 
# copy warning, cuts down on memory usage by not assigning intermediate variables and is easily 
# reproducable with one funtion call.  

In [38]:
import datetime
import pandas as pd
import os
import numpy as np

In [39]:
path = os.path.join(os.getcwd(), "blockchain_explorer_outputs")
immutable = pd.read_csv(os.path.join(path, 'gamestop_wallet_immutable.csv'))
immutable.txn_type.value_counts()

transfer    130
buy          81
mint         77
deposit       9
sell          1
Name: txn_type, dtype: int64

In [40]:
immutable.columns

Index(['txn_id', 'txn_time', 'txn_type', 'from_address', 'to_address',
       'token_address', 'collection_name', 'collection_image_uri', 'token_id',
       'token_name', 'token_image_uri', 'token_symbol', 'token_quantity',
       'usd_amount'],
      dtype='object')

In [41]:
immutable.head()

,txn_id,txn_time,txn_type,from_address,to_address,token_address,collection_name,collection_image_uri,token_id,token_name,token_image_uri,token_symbol,token_quantity,usd_amount
0,124991094,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056767.0,Lootable Corpse,https://card.godsunchained.com/?id=1681&q=3,NaN,NaN,NaN
1,124991095,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056768.0,On Her Command,https://card.godsunchained.com/?id=1609&q=4,NaN,NaN,NaN
2,124991096,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056769.0,Surpassing Blast,https://card.godsunchained.com/?id=1527&q=4,NaN,NaN,NaN
3,124991097,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056770.0,Engaged Healer,https://card.godsunchained.com/?id=1504&q=4,NaN,NaN,NaN
4,124991098,2022-10-08T23:02:15.946Z,mint,NaN,NaN,0xacb3c6a43d15b907e8433077b6d38ae40936fe2c,Gods Unchained Cards,https://images.godsunchained.com/misc/gu-sigel...,211056771.0,Ash Reader,https://card.godsunchained.com/?id=1535&q=4,NaN,NaN,NaN


In [42]:
def tweak_dataframe(df_ = immutable):
    """Converts immutable csv dataframe into a form acceptable by koinly tax software"""
    def combine_columns(df_, columns:list):
        """Function to combine calculated fields for specific transaction types"""
        series = df_[columns[0]]
        del columns[0]
        while len(columns) > 0:
            series = series.combine_first(other=df_[columns[0]])
            del columns[0]
        return series

    return (df_
    .assign(Date=lambda df_:
        pd.to_datetime(df_.txn_time).dt.strftime('%Y-%m-%d %H:%M %Z'))
    # dealing with the buy transaction types
    .assign(Sent_Amount_Buy=lambda df_: 
        df_.token_quantity.where(df_.txn_type=='buy', np.nan))
    .assign(Sent_Currency_Buy=lambda df_: 
        df_.token_symbol.where(df_.txn_type=='buy', np.nan))
    .assign(Received_Amount_Buy=lambda df_:
        np.where(df_.txn_type=='buy', 1, np.nan))
    .assign(Received_Currency_Buy=lambda df_:
        df_.token_id.where(df_.txn_type=='buy', np.nan))
    # dealing with sell transaction types
    .assign(Received_Amount_Sell=lambda df_: 
        df_.token_quantity.where(df_.txn_type=='sell', np.nan))
    .assign(Received_Currency_Sell=lambda df_: 
        df_.token_symbol.where(df_.txn_type=='sell', np.nan))
    .assign(Sent_Amount_Sell=lambda df_:
        np.where(df_.txn_type=='sell', 1, np.nan))
    .assign(Sent_Currency_Sell=lambda df_:
        df_.token_id.where(df_.txn_type=='sell', np.nan))
    # dealing with mint transaction types
    .assign(Received_Amount_Mint=lambda df_:
        np.where(df_.txn_type=='mint', 1, np.nan))
    .assign(Received_Currency_Mint=lambda df_:
        df_.token_id.where(df_.txn_type=='mint', np.nan))
    # dealing with transfer transaction types
    #   First, dealing with transfers sent from the wallet
    .assign(Sent_Amount_Transfer=lambda df_:
        df_.token_quantity.where(
            (df_.txn_type=="transfer")&(~df_.to_address.isna())))
    .assign(Sent_Currency_Transfer=lambda df_:
        df_.token_symbol.where(
            (df_.txn_type=="transfer")&(~df_.to_address.isna())))
    #   Next, dealing with transfers received by the wallet
    .assign(Received_Amount_Transfer=lambda df_:
        df_.token_quantity.where(
            (df_.txn_type=="transfer")&(~df_.from_address.isna())))
    .assign(Received_Currency_Transfer=lambda df_:
        df_.token_symbol.where(
            (df_.txn_type=="transfer")&(~df_.from_address.isna())))
    # dealing with deposit transaction types
    .assign(Received_Amount_Deposit=lambda df_:
        df_.token_quantity.where(df_.txn_type=="deposit",np.nan))
    .assign(Received_Currency_Deposit=lambda df_:
        df_.token_symbol.where(df_.txn_type=="deposit", np.nan))
    # Merging together applicable columns
    .assign(Sent_Amount=lambda df_:
        combine_columns(df_, columns=[
        'Sent_Amount_Buy', 'Sent_Amount_Transfer', 'Sent_Amount_Sell']))
    .assign(Sent_Currency=lambda df_:
        combine_columns(df_, columns=[
        'Sent_Currency_Buy', 'Sent_Currency_Transfer', 'Sent_Currency_Sell']))
    .assign(Received_Amount=lambda df_:
        combine_columns(df_, columns=[
        'Received_Amount_Sell', 'Received_Amount_Transfer', 'Received_Amount_Deposit', 
        'Received_Amount_Mint']))
    .assign(Received_Currency=lambda df_:
        combine_columns(df_, columns=[
        'Received_Currency_Sell', 'Received_Currency_Transfer', 'Received_Currency_Deposit',
        'Received_Currency_Mint']))
    # Creating Net_Worth_Currency column
    .assign(Net_Worth_Currency = 'USD')
    # renaming columns that need no processing
    .rename(columns={'txn_id':'TxHash', 'txn_type':'Label', 'usd_amount':'Net Worth Amount'})
    # dropping unneeded columns
    .drop(columns=['txn_time', 'token_address', 'collection_name', 'collection_image_uri', 
                   'token_id', 'token_image_uri', 'Sent_Amount_Buy', 'Sent_Currency_Buy',
                   'Received_Amount_Sell', 'Received_Currency_Sell', 'Received_Amount_Transfer',
                   'Received_Currency_Transfer', 'Sent_Amount_Transfer', 'Sent_Currency_Transfer',
                   'Received_Amount_Deposit', 'Received_Currency_Deposit', 'Received_Amount_Buy',
                   'Received_Currency_Buy', 'Sent_Amount_Sell', 'Sent_Currency_Sell',
                   'Received_Amount_Mint', 'Received_Currency_Mint'])
    # Changing all column name underscores to spaces
    .rename(columns=lambda c: c.replace('_', ' ')))

In [43]:
df = tweak_dataframe(immutable)

In [44]:
df.columns

Index(['TxHash', 'Label', 'from address', 'to address', 'token name',
       'token symbol', 'token quantity', 'Net Worth Amount', 'Date',
       'Sent Amount', 'Sent Currency', 'Received Amount', 'Received Currency',
       'Net Worth Currency'],
      dtype='object')